In [1]:
import re
import mysql.connector
import numpy as np

In [2]:
mydb = mysql.connector.connect(
    host='127.0.0.1', 
    user='Yuxin Xiao', 
    password='19961002XYX',
    database='CS510Project')
mycursor = mydb.cursor()

In [44]:
mycursor.execute("CREATE TABLE NEW_Sentences (Sentence_id INT, Sentence TEXT)")
mycursor.execute("CREATE TABLE NEW_Words (Word TEXT, Count INT, Inverted_index LONGTEXT)")

In [43]:
mycursor.execute("DROP TABLE NEW_Sentences")
mycursor.execute("DROP TABLE NEW_Words")

In [45]:
def insert_sentences(Id, Sentence):
    sql = ('INSERT INTO NEW_Sentences '
           '(Sentence_id, Sentence) '
           'VALUES (%s, %s)')
    val = (Id, Sentence)
    mycursor.execute(sql, val)

In [46]:
with open("sentences-remove-zeros.txt", "r", encoding='utf-8', errors='ignore') as file:
    sentences = file.read().split('\n')
    for Id, Sentence in enumerate(sentences):
        insert_sentences(Id, Sentence)
        
        if Id % 5000 == 0:
            print(Id)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000


In [47]:
mycursor.execute("SELECT * FROM NEW_Sentences")
myresult = mycursor.fetchall()

In [51]:
print("Creating Dictionary")
print(len(myresult))
   
count_dict = {}
index_dict = {}

num = 0
for row in myresult:
    
    num += 1
    if num % 5000 == 0:
        print(num)
    
    index = str(row[0])
    sentence = re.sub(r'[^\w\s]', "", row[1]).split()
        
    for word in sentence:
        word = word.lower()
        
        if word in count_dict:
            count_dict[word] += 1
            index_dict[word] += (' ' + index)
        else:
            count_dict[word] = 1
            index_dict[word] = index

Creating Dictionary
58148
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000


In [53]:
print("Writing into Database")
print(len(count_dict.keys()))

num = 0
for word in count_dict.keys():
    
    num += 1
    if num % 5000 == 0:
        print(num)
    
    count = count_dict[word]
    index = index_dict[word]

    insert_sql = "INSERT INTO NEW_Words (Word, Count, Inverted_index) VALUES (%s, %s, %s)"
    insert_val = (word, count, index)
    mycursor.execute(insert_sql, insert_val)

Writing into Database
78812
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000


In [54]:
mycursor.execute("SELECT * FROM NEW_Words")
myresult = mycursor.fetchall()

In [57]:
mydb.commit()

In [60]:
mycursor.close()
mydb.close()